**thermanalyze_b300**

This function analyzes the input image and makes a diagnosis.

In [0]:
def thermanalyze_b300(input_image,input_temp_max,input_temp_min, pixel_step = 1, tolerance = 5):
  #Part 1 - Defining tic and toc functions
  def tic():
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()
  def toc():
    import time
    if 'startTime_for_tictoc' in globals():
      print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
      print("Toc: start time not set")
  #Part 2 - Checking bounds temperature
  tic()
  if input_temp_max < input_temp_min:
    input_temp_change = input_temp_max
    input_temp_max = input_temp_min
    input_temp_min = input_temp_change
  #Part 3 - Image reading
  import cv2
  image_color_data = cv2.imread(input_image,1)
  reference_color_data = image_color_data[61:177,297:305,:]
  #Part 4 - Temperature reference
  import numpy as np
  reference_max_pixel = len(reference_color_data)
  reference_temperature = np.zeros(shape = (reference_max_pixel))
  for t in range(0,reference_max_pixel):
    reference_temperature[t] = input_temp_max - (input_temp_max-input_temp_min)*((t)/float(reference_max_pixel-1))
  #Part 5 - Temperature image
  thermal_xmax = len(image_color_data)
  thermal_ymax = int(image_color_data.size/(3*thermal_xmax))
  cond_color = [0,0,0]
  thermal_temperature = np.zeros(shape = (thermal_xmax, thermal_ymax))
  thermal_temperature[:] = np.nan
  while thermal_xmax % pixel_step + thermal_ymax % pixel_step != 0:
    pixel_step = pixel_step - 1
  reference_max_pixel_c = int(reference_color_data.size/(reference_max_pixel*3))
  for reference_pixel_c in range(0,reference_max_pixel_c):
    for thermal_x in range(0,thermal_xmax,pixel_step):
      for thermal_y in range(0,thermal_ymax,pixel_step):
        if np.isnan(thermal_temperature[thermal_x,thermal_y]) == True:
          for reference_pixel in range(0,reference_max_pixel):
            for count_color in range(0,3):
              if image_color_data[thermal_x,thermal_y,count_color] > reference_color_data[reference_pixel,reference_pixel_c,count_color]:
                if image_color_data[thermal_x,thermal_y,count_color] - reference_color_data[reference_pixel,reference_pixel_c,count_color] <= tolerance:
                  cond_color[count_color] = 1;
                else:
                  cond_color[count_color] = 0;
              else:
                if reference_color_data[reference_pixel,reference_pixel_c,count_color] - image_color_data[thermal_x,thermal_y,count_color] <= tolerance:
                  cond_color[count_color] = 1;
                else:
                  cond_color[count_color] = 0;
            if cond_color[0] == 1 and cond_color[1] == 1 and cond_color[2] == 1:
              thermal_temperature[thermal_x,thermal_y] = reference_temperature[reference_pixel]
  #Part 6 - Comparing the input with the library
  import pandas as pd
  import os.path
  import csv
  count_score = 0;
  file_score = 0;
  ini_lock = 0;
  for file_count in range(0,10000):
    if os.path.isfile('FLIR_B300_'+str("%0.4d"%file_count)+'.csv'):
      file_data = pd.read_csv('FLIR_B300_'+str("%0.4d"%file_count)+'.csv', header = None)
      for count_x in range(0,thermal_xmax):
        for count_y in range(0,thermal_ymax):
          if abs(file_data.loc[count_x,count_y]-thermal_temperature[count_x,count_y]) < tolerance*(input_temp_max-input_temp_min)/(reference_max_pixel-1):
            file_score = file_score + 1/(thermal_xmax*thermal_ymax)
      if ini_lock == 1:
        if file_score > val_best_score:
          val_best_score = file_score
          code_best_score = file_count
        result.append([file_count,file_score])
      else:
        val_best_score = file_score
        code_best_score = file_count
        result = [[file_count,file_score]]
        ini_lock = 1
      count_score = count_score + 1
      file_score = 0
  #Part 7 - Results
  result = np.array(result)
  import matplotlib.pyplot as plt
  plt.bar(list(range(1,len(result[:,0])+1)),result[:,1])
  plt.xlabel('Library File')
  plt.ylabel('Compatibility')
  diagnosis_best_score = int(code_best_score/10)
  ReadExcel = pd.read_excel('diagnosis_list.xlsx')
  print(ReadExcel.loc[diagnosis_best_score,'diagnosis'])
  print('probability: '+str(val_best_score*100)+'%')
  toc()